In [1]:
.libPaths()
library(biomaRt)
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")


[1] "/disk1/wenqing/anaconda3/envs/wq_R4.2/lib/R/library"

In [8]:

#gene_ids <- read.table("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/de_editing/GABAergic_neurons/genes_with_de_edited_res.txt")[,1]
gene_ids <- read.table("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/editing_level_change/notwith1/cor_threshold_0.3/top20_genes_related2pc1.txt")[,1]
length(gene_ids)


[1] 14

In [4]:
attributes <- c("ensembl_gene_id_version","ensembl_gene_id", "external_gene_name", "description", "chromosome_name", "start_position", "end_position","gene_biotype")
results <- getBM(attributes = attributes, filters = "ensembl_gene_id_version", values = gene_ids, mart = ensembl)
results$chromosome_name <- paste0("chr",results$chromosome_name)
dim(results)

,ensembl_gene_id_version,ensembl_gene_id,external_gene_name,description,chromosome_name,start_position,end_position,gene_biotype
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
1,ENSG00000002746.15,ENSG00000002746,HECW1,"HECT, C2 and WW domain containing E3 ubiquitin protein ligase 1 [Source:HGNC Symbol;Acc:HGNC:22195]",chr7,43112629,43566001,protein_coding
2,ENSG00000100325.15,ENSG00000100325,ASCC2,activating signal cointegrator 1 complex subunit 2 [Source:HGNC Symbol;Acc:HGNC:24103],chr22,29788609,29838304,protein_coding
3,ENSG00000123815.13,ENSG00000123815,COQ8B,coenzyme Q8B [Source:HGNC Symbol;Acc:HGNC:19041],chr19,40691514,40725784,protein_coding
4,ENSG00000158864.13,ENSG00000158864,NDUFS2,NADH:ubiquinone oxidoreductase core subunit S2 [Source:HGNC Symbol;Acc:HGNC:7708],chr1,161197104,161214723,protein_coding
5,ENSG00000166272.18,ENSG00000166272,WBP1L,WW domain binding protein 1 like [Source:HGNC Symbol;Acc:HGNC:23510],chr10,102743948,102834516,protein_coding
6,ENSG00000215196.5,ENSG00000215196,BASP1-AS1,BASP1 antisense RNA 1 [Source:HGNC Symbol;Acc:HGNC:26609],chr5,17089296,17217047,lncRNA


In [9]:
length(results$external_gene_name)
write.table(unique(results$external_gene_name),"/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/editing_level_change/notwith1/cor_threshold_0.3/top20_genes_related2pc1_geneNameVer.txt",quote=F,col.names=F,row.names=F)


[1] 8

In [ ]:
length(paste0(results$external_gene_name,"_",results$gene_biotype))
length(unique(paste0(results$external_gene_name,"_",results$gene_biotype)))
results[order(results$external_gene_name),]



In [ ]:
length(results$external_gene_name)
length(unique(results$external_gene_name))
length(paste0(results$external_gene_name,"_",results$gene_biotype))
length(unique(paste0(results$external_gene_name,"_",results$gene_biotype)))
results[order(results$external_gene_name),]



In [ ]:
##在每个富集到res的基因后面备注该基因的类型
info <- read.table("/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/de_editing/GABAergic_neurons/detailed_info_genes_with_de_edited_res.txt")
head(info)

In [61]:
###注意：一个基因可能有多个基因id，对应不同的类型，在这个部分的分析中，我们去除了有多重gene biotype的gene
#仅保留1:1的gene
all_gene_types <- c()
for(i in 1:nrow(info)){
    res <- unique(results$gene_biotype[results$external_gene_name==info$V4[i]])
    if(length(res)==1){
        gene_type <- res
    }else{
        gene_type <- 'unKnown'
    }
    all_gene_types <- c(all_gene_types,gene_type)
}

In [63]:
info$V9 <- all_gene_types

In [64]:
write.table(info,"/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/de_editing/GABAergic_neurons/detailed_info_genes_with_de_edited_res_with_geneType.txt",quote=F,sep='\t',col.names=F,row.names=F)

In [69]:
##统计各类型基因intron富集的res数量
table(info$V9)


                            lncRNA                     protein_coding 
                                17                                466 
                             scRNA transcribed_unprocessed_pseudogene 
                                 1                                  8 
                           unKnown             unprocessed_pseudogene 
                               156                                  1 

[1] 17

[1] 17

In [70]:
lncRNA_dat <- info[which(info$V9=='lncRNA'),]
length(unique(paste0(lncRNA_dat$V6,"_",lncRNA_dat$V8)))
length(paste0(lncRNA_dat$V6,"_",lncRNA_dat$V8))
mRNA_dat <- info[which(info$V9=='protein_coding'),]
length(unique(paste0(mRNA_dat$V6,"_",mRNA_dat$V8)))

[1] 17

[1] 17

[1] 428

[1] 466

In [80]:
attributes_for_GO <- c("ensembl_gene_id_version","ensembl_gene_id", "external_gene_name")
lncRNA_ESG <- getBM(attributes = attributes_for_GO, filters = "external_gene_name", values = unique(lncRNA_dat$V4), mart = ensembl)
write.table(lncRNA_ESG,"/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/de_editing/GABAergic_neurons/lncRNA_ESG.txt",quote=F,col.names=F,row.names=F,sep='\t')

In [76]:
write.table(unique(lncRNA_dat$V4),"/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/de_editing/GABAergic_neurons/lncRNA_genes_with_de_edited_sites.txt",quote=F,col.names=F,row.names=F)
write.table(unique(mRNA_dat$V4),"/disk1/wenqing/tmp_data/PFC_s2/all_analysis_result/de_editing/GABAergic_neurons/mRNA_genes_with_de_edited_sites.txt",quote=F,col.names=F,row.names=F)

In [73]:
1313/74
2133/263

2019/95
3466/321

[1] 17.74324

[1] 8.110266

[1] 21.25263

[1] 10.79751

In [21]:
####分别提取lncRNA、mRNA、miRNA形成region的bed文件
region_lncRNA <- results[which(results$gene_biotype=='lncRNA'),c('chromosome_name','start_position','end_position','external_gene_name')]
region_mRNA <- results[which(results$gene_biotype=='protein_coding'),c('chromosome_name','start_position','end_position','external_gene_name')]
region_miRNA <- results[which(results$gene_biotype=='miRNA'),c('chromosome_name','start_position','end_position','external_gene_name')]

In [27]:
region_lncRNA$external_gene_name[which(region_lncRNA$external_gene_name=='')] <- 'No_name'
region_mRNA$external_gene_name[which(region_mRNA$external_gene_name=='')] <- 'No_name'
region_miRNA$external_gene_name[which(region_miRNA$external_gene_name=='')] <- 'No_name'

In [16]:
table(region_lncRNA$external_gene_name!='')


FALSE  TRUE 
 5729  1883 

In [17]:
#######lncRNA, mRNA和miRNA有些没有对应的基因名称，无法计算相应的IRI，因此去掉这部份分子
region_lncRNA <- region_lncRNA[which(region_lncRNA$external_gene_name!=''),]
region_mRNA <- region_mRNA[which(region_mRNA$external_gene_name!=''),]
region_miRNA <- region_miRNA[which(region_miRNA$external_gene_name!=''),]

In [18]:
dim(region_lncRNA)
dim(region_miRNA)
dim(region_mRNA)

[1] 1883    4

[1] 991   4

[1] 348   4

In [28]:
write.table(region_lncRNA,"/disk1/wenqing/tmp_data/lncRNA.withNoname.bed",quote=F,row.names=F,col.names=F,sep='\t')
write.table(region_mRNA,"/disk1/wenqing/tmp_data/mRNA.withNoname.bed",quote=F,row.names=F,col.names=F,sep='\t')
write.table(region_miRNA,"/disk1/wenqing/tmp_data/miRNA.withNoname.bed",quote=F,row.names=F,col.names=F,sep='\t')

In [ ]:
write.table(results,"/disk1/wenqing/tmp_data/uniq_gencodeNONCODE_gene_info.txt",quote=F,row.names=F,sep='\t')

In [19]:
listAttributes(ensembl)

name,description,page
<chr>,<chr>,<chr>
ensembl_gene_id,Gene stable ID,feature_page
ensembl_gene_id_version,Gene stable ID version,feature_page
ensembl_transcript_id,Transcript stable ID,feature_page
ensembl_transcript_id_version,Transcript stable ID version,feature_page
ensembl_peptide_id,Protein stable ID,feature_page
ensembl_peptide_id_version,Protein stable ID version,feature_page
ensembl_exon_id,Exon stable ID,feature_page
description,Gene description,feature_page
chromosome_name,Chromosome/scaffold name,feature_page
